In [1]:
from pathlib import Path
from functools import reduce
import numpy as np

In [2]:
def load_puzzle_input(path):
    puzzle_input_path = Path(path)
    warehouse_map = []
    moves = []
    moves_mapping = { '>': np.array([0, 1]), '^': np.array([-1, 0]), '<': np.array([0, -1]), 'v': np.array([1, 0])}

    with open(puzzle_input_path, 'r') as f:
        is_move_line = False
        for row, line in enumerate(f):
            line = line.strip()
            if row == 0:
                continue
            if line == ('#' * len(line)):
                is_move_line = True
                continue
            if not is_move_line:
                warehouse_map.append(list(line[1:-1]))
            else:
                moves.extend([moves_mapping[move] for move in line])
    return np.array(warehouse_map), moves

In [3]:
class WharehouseMap:
    def __init__(self, warehouse_map):
        self.warehouse_map = warehouse_map.copy()
        self.shape_array = np.array(self.warehouse_map.shape)

    def move(self, position: np.ndarray, move: np.ndarray):
        new_position = position + move
        if ((new_position < 0) | (new_position >= self.shape_array)).any():
            return False
        idx_position = tuple(position)
        idx_new_position = tuple(new_position)
        match self.warehouse_map[idx_new_position]:
            case '.':
                self.warehouse_map[idx_position], self.warehouse_map[idx_new_position] = self.warehouse_map[idx_new_position], self.warehouse_map[idx_position]
                return True
            case '#':
                return False
            case _:
                if self.move(new_position, move):
                    self.warehouse_map[idx_position], self.warehouse_map[idx_new_position] = self.warehouse_map[idx_new_position], self.warehouse_map[idx_position]
                    return True
                return False

In [4]:
warehouse_map, moves = load_puzzle_input('inputs/day15.txt')
robot = np.where(warehouse_map == '@')
robot_position = np.array(list(zip(robot[0], robot[1]))[0])
move_map = WharehouseMap(warehouse_map)
for move in moves:
    if move_map.move(robot_position, move):
        robot_position += move

In [5]:
indices = np.where(move_map.warehouse_map == 'O')
reduce(lambda acc, value: acc + 100 * (value[0] + 1) + (value[1] + 1), zip(indices[0], indices[1]), 0)


1448589